**Note:** This interactive dashboard only works when run in a Jupyter environment with ipywidgets installed. The widgets will not be visible in GitHub's preview, but screenshots are attached for reference.


In [3]:
# Step 8: Create the Interactive Dashboard

# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle  # Make sure this is imported
import ipywidgets as widgets
from IPython.display import display, HTML

# Load the model
try:
    with open('telco_churn_model.pkl', 'rb') as file:
        model_components = pickle.load(file)
    
    pipeline = model_components['pipeline']
    categorical_cols = model_components['categorical_cols']
    numerical_cols = model_components['numerical_cols']
    feature_cols = model_components['feature_cols']
    
    print("Model loaded successfully!")
except FileNotFoundError:
    print("Model file not found. Please run the previous cells to train and save the model first.")
except Exception as e:
    print(f"Error loading model: {str(e)}")

Model loaded successfully!


In [4]:
# Interactive Dashboard for Churn Prediction
import ipywidgets as widgets
from IPython.display import display, HTML

def create_dashboard(model_components):
    # Load model components
    pipeline = model_components['pipeline']
    categorical_cols = model_components['categorical_cols']
    numerical_cols = model_components['numerical_cols']
    
    # Create widgets for each feature
    feature_widgets = {}
    
    # Create numerical widgets
    for col in numerical_cols:
        if col == 'tenure':
            feature_widgets[col] = widgets.IntSlider(
                value=24, min=0, max=72, step=1, description=f'{col}:',
                style={'description_width': 'initial'}, layout=widgets.Layout(width='500px')
            )
        elif col == 'MonthlyCharges':
            feature_widgets[col] = widgets.FloatSlider(
                value=65.0, min=10.0, max=150.0, step=1.0, description=f'{col}:',
                style={'description_width': 'initial'}, layout=widgets.Layout(width='500px')
            )
        elif col == 'TotalCharges':
            feature_widgets[col] = widgets.FloatSlider(
                value=1500.0, min=0.0, max=9000.0, step=100.0, description=f'{col}:',
                style={'description_width': 'initial'}, layout=widgets.Layout(width='500px')
            )
        else:
            feature_widgets[col] = widgets.FloatSlider(
                value=0.0, min=0.0, max=10.0, step=0.1, description=f'{col}:',
                style={'description_width': 'initial'}, layout=widgets.Layout(width='500px')
            )
    
    # Create categorical widgets
    categorical_values = {
        'gender': ['Female', 'Male'],
        'Partner': ['Yes', 'No'],
        'Dependents': ['Yes', 'No'],
        'PhoneService': ['Yes', 'No'],
        'MultipleLines': ['No', 'Yes', 'No phone service'],
        'InternetService': ['DSL', 'Fiber optic', 'No'],
        'OnlineSecurity': ['No', 'Yes', 'No internet service'],
        'OnlineBackup': ['No', 'Yes', 'No internet service'],
        'DeviceProtection': ['No', 'Yes', 'No internet service'],
        'TechSupport': ['No', 'Yes', 'No internet service'],
        'StreamingTV': ['No', 'Yes', 'No internet service'],
        'StreamingMovies': ['No', 'Yes', 'No internet service'],
        'Contract': ['Month-to-month', 'One year', 'Two year'],
        'PaperlessBilling': ['Yes', 'No'],
        'PaymentMethod': ['Electronic check', 'Mailed check', 'Bank transfer (automatic)', 'Credit card (automatic)']
    }
    
    for col in categorical_cols:
        if col in categorical_values:
            feature_widgets[col] = widgets.Dropdown(
                options=categorical_values[col],
                value=categorical_values[col][0],
                description=f'{col}:',
                style={'description_width': 'initial'},
                layout=widgets.Layout(width='500px')
            )
        else:
            feature_widgets[col] = widgets.Dropdown(
                options=['Yes', 'No'],
                value='No',
                description=f'{col}:',
                style={'description_width': 'initial'},
                layout=widgets.Layout(width='500px')
            )
    
    # Function to predict churn
    def predict_churn(button):
        # Create a dataframe with user inputs
        input_data = {}
        for col, widget in feature_widgets.items():
            input_data[col] = [widget.value]
        
        input_df = pd.DataFrame(input_data)
        
        # If TotalCharges widget value is less than MonthlyCharges, adjust it
        if input_df['TotalCharges'].iloc[0] < input_df['MonthlyCharges'].iloc[0]:
            input_df['TotalCharges'] = input_df['MonthlyCharges']
        
        # Perform feature engineering on input data
        # For simplicity, we're just calculating ServiceCount here
        service_cols = ['PhoneService', 'MultipleLines', 'OnlineSecurity', 'OnlineBackup', 
                       'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']
        
        input_df['ServiceCount'] = 0
        for col in service_cols:
            if col in input_df.columns:
                input_df['ServiceCount'] += (input_df[col] == 'Yes').astype(int)
        
        input_df['HasInternetService'] = (input_df['InternetService'] != 'No').astype(int)
        
        # Make prediction
        try:
            prediction_proba = pipeline.predict_proba(input_df)[0, 1]
            prediction = pipeline.predict(input_df)[0]
            
            # Show results
            if prediction == 1:
                result_html = f"""
                <div style="background-color:#FFCCCC; padding:15px; border-radius:10px; margin-top:10px;">
                    <h2 style="color:#CC0000; margin:0;">High Churn Risk: {prediction_proba:.2%}</h2>
                    <p>This customer is likely to churn. Consider targeted retention actions.</p>
                </div>
                """
            else:
                result_html = f"""
                <div style="background-color:#CCFFCC; padding:15px; border-radius:10px; margin-top:10px;">
                    <h2 style="color:#006600; margin:0;">Low Churn Risk: {prediction_proba:.2%}</h2>
                    <p>This customer is likely to stay. Continue providing good service.</p>
                </div>
                """
            
            result_output.value = result_html
            
            # Show feature importance for this prediction (if applicable)
            if hasattr(pipeline.named_steps['model'], 'feature_importances_'):
                feature_imp_output.value = "Feature importance visualization not implemented in this version of the dashboard."
            else:
                feature_imp_output.value = ""
                
        except Exception as e:
            result_output.value = f"<div style='color:red'>Error making prediction: {str(e)}</div>"
    
    # Create predict button
    predict_button = widgets.Button(
        description='Predict Churn',
        button_style='primary',
        layout=widgets.Layout(width='200px')
    )
    predict_button.on_click(predict_churn)
    
    # Output widgets for results
    result_output = widgets.HTML("")
    feature_imp_output = widgets.HTML("")
    
    # Create sections
    demographics_box = widgets.VBox([
        widgets.HTML("<h3>Demographics</h3>"),
        feature_widgets['gender'],
        feature_widgets['Partner'],
        feature_widgets['Dependents']
    ])
    
    account_box = widgets.VBox([
        widgets.HTML("<h3>Account Information</h3>"),
        feature_widgets['tenure'],
        feature_widgets['Contract'],
        feature_widgets['PaperlessBilling'],
        feature_widgets['PaymentMethod'],
        feature_widgets['MonthlyCharges'],
        feature_widgets['TotalCharges']
    ])
    
    services_box = widgets.VBox([
        widgets.HTML("<h3>Services</h3>"),
        feature_widgets['PhoneService'],
        feature_widgets['MultipleLines'],
        feature_widgets['InternetService'],
        feature_widgets['OnlineSecurity'],
        feature_widgets['OnlineBackup'],
        feature_widgets['DeviceProtection'],
        feature_widgets['TechSupport'],
        feature_widgets['StreamingTV'],
        feature_widgets['StreamingMovies']
    ])
    
    # Layout the dashboard
    dashboard = widgets.VBox([
        widgets.HTML("<h1>Telco Customer Churn Predictor</h1>"),
        widgets.HTML("<p>Adjust the parameters below to predict customer churn probability.</p>"),
        widgets.HBox([demographics_box, account_box, services_box]),
        predict_button,
        result_output,
        feature_imp_output
    ])
    
    return dashboard

# Try to load the saved model
try:
    with open('telco_churn_model.pkl', 'rb') as file:
        model_components = pickle.load(file)
    
    # Create and display dashboard
    dashboard = create_dashboard(model_components)
    display(dashboard)
    
    print("Dashboard created successfully! Adjust the parameters and click 'Predict Churn'.")
    
except FileNotFoundError:
    print("Model file not found. Please run the previous cells to train and save the model first.")
except Exception as e:
    print(f"Error loading model: {str(e)}")

Dashboard created successfully! Adjust the parameters and click 'Predict Churn'.


## Dashboard Demo
![Dashboard Demo](churn_low_risk_demo.gif)
*GIF demonstration of the interactive dashboard features*